In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001BABFA94440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BABFA94AD0>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage,SystemMessage
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello how are you?")
]

result=model.invoke(messages) 

In [6]:
result

AIMessage(content='Bonjour, comment ça va\u202f?', additional_kwargs={'reasoning_content': 'We need to translate "Hello how are you?" to French. Likely "Bonjour, comment ça va ?" or "Bonjour, comment vas-tu ?" The user just asked to translate. So output the translation.'}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 86, 'total_tokens': 145, 'completion_time': 0.122006742, 'completion_tokens_details': {'reasoning_tokens': 43}, 'prompt_time': 0.003234742, 'prompt_tokens_details': None, 'queue_time': 0.049845938, 'total_time': 0.125241484}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_626f3fc5e0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b66fc-8f7f-70e3-86b1-068377cf4979-0', usage_metadata={'input_tokens': 86, 'output_tokens': 59, 'total_tokens': 145, 'output_token_details': {'reasoning': 43}})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment ça va\u202f?'

In [8]:
### using LCEL we can chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment ça va\u202f?'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [10]:
prompt.invoke({"language":"French","text":"Hello"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [12]:
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour'